In [59]:
import csv
import os
import sys
# Spark imports
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import desc
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, udf, array_contains
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql.types import ArrayType, StringType, FloatType, IntegerType
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import matplotlib.pyplot as plt

from csv import reader
from collections import defaultdict

In [2]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

<Font size=5 color=red>Investigate the original dataset (obviously, it cannot be used). Take a look at https://stackoverflow.com/questions/13793529/r-error-invalid-type-list-for-variable to see how useless the Body column information could be!

The point here is that the body information consists mostly of codes and some weird patterns that are not useful for our purpose. The most important information here is the connection between the title of the questions and tags. So, I removed the Body column from the dataset.</Font>

In [3]:
try:
    spark = init_spark()

    filename1 = "./Train.csv"
    df2 = spark.read.option("multiLine", 'true').option("escape","\'").csv(filename1, header=True)
    print(df2.count())
    print(df2.show(10))    
except:
    pass

<Font size=5 color=red>For removing the Body column, I read all the dataset once using Pandas library. After that, I removed the column and got an export to have a concrete file as our dataset. This part has been ommited from the notebook.</Font>

In [4]:
spark = init_spark()

filename = "./TrainWithoutBody.csv"
df1 = spark.read.option("multiLine", 'true').option("escape","\'").csv(filename, header=True)
df1 = df1.drop("_c0")
df1 = df1.dropna()

rddTags = df1.select("Tags").rdd

# df1.count()

In [5]:
# df1.show(5)

<Font size = 5, color=green>Finding the 100 most used tags (one DT per each most used tag)

In [6]:
splittedTags = rddTags.filter(lambda r: r[0] != None).flatMap(lambda r: r[0].split(" ")).map(lambda r: r.replace(".", "")).map(lambda r: (r, 1)).reduceByKey(lambda x, y: x + y)

splittedTags = splittedTags.sortBy(lambda r: r[1], False) #Sorted with number of usage (you can collect and see)

splittedTagsSorted = splittedTags.map(lambda r: r[0]) #Delete this line if you want to see number of times they have been used.

# df10 = anSorted.toDF()


mostUsedTags = splittedTagsSorted.collect()[0:50]



In [7]:
mostUsedTags

['c#',
 'java',
 'php',
 'javascript',
 'android',
 'jquery',
 'c++',
 'python',
 'iphone',
 'aspnet',
 'mysql',
 'html',
 'net',
 'ios',
 'objective-c',
 'sql',
 'css',
 'linux',
 'ruby-on-rails',
 'windows',
 'c',
 'sql-server',
 'ruby',
 'wpf',
 'xml',
 'ajax',
 'database',
 'regex',
 'windows-7',
 'aspnet-mvc',
 'xcode',
 'django',
 'osx',
 'arrays',
 'vbnet',
 'eclipse',
 'json',
 'facebook',
 'ruby-on-rails-3',
 'ubuntu',
 'performance',
 'networking',
 'string',
 'multithreading',
 'winforms',
 'security',
 'aspnet-mvc-3',
 'visual-studio-2010',
 'bash',
 'homework']

In [8]:
# print(rddTags.count())

In [9]:
# rddTags.take(10)

<Font size=5.5, color="purpule">Here, I have cleaned the Tags column to only contain the most used tags. For example, I ommited the "upload" tag from first group of tags for the first question, because it's not a most used tag.

In [10]:
def replaceNoneWithString(x):
    if (x == None): return "None"
    else : return x

# Subject titles to TF-IDF

In [11]:
tokenizer = Tokenizer(inputCol="Title", outputCol="transformed_tfidf")
wordsData = tokenizer.transform(df1)

hashingTF = HashingTF(inputCol="transformed_tfidf", outputCol="rawFeatures", numFeatures=100)
featurizedData = hashingTF.transform(wordsData)

featurizedData.take(1)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [12]:
# rescaledData.select("id", "title", "features").show()

## Clean up tags to include only most-used tags

In [13]:
def cleanUpTags(tags):
    tags = tags.split(" ")
    tags = [tag for tag in tags if tag in mostUsedTags]
    return tags

cleanUpTagsUDF = udf(cleanUpTags, ArrayType(StringType()))


df = rescaledData.withColumn("cleantags",  cleanUpTagsUDF(col("tags")))

In [14]:
def filterEmptyRows(tags):
    return len(tags)

filterEmptyRowsUDF = udf(filterEmptyRows, IntegerType())

df = df.filter(filterEmptyRowsUDF(col("cleantags")) > 0)
# df.select("id", "title", "features", "cleantags").show()
df = df.sample(0.00083, 42)
df.count()

3195

In [15]:
for tag in mostUsedTags:
    df = df.withColumn(tag, (array_contains(df.cleantags, tag)).cast('integer'))

# df.select("Javascript").show()

# Split data to test, train and validation sets

In [16]:
train_data, testVal_data = df.randomSplit([.7,.3],seed=1234)
test_data, validation_data = testVal_data.randomSplit([.5, .5], seed=1234)

In [17]:
# test_data.select("features").show()

# Train DecisionTreeClassifiers

In [18]:
dtcs = defaultdict()
preds = defaultdict()
for tag in mostUsedTags:
    if tag not in dtcs:
        dtcs[tag] = DecisionTreeClassifier(featuresCol="features", labelCol=tag, maxDepth=2).fit(train_data)
#         preds[tag] = dtcs[tag].transform(validation_data)

# predict later
# dtc = DecisionTreeClassifier(featuresCol="features", labelCol="javascript", maxDepth=2)

In [19]:
# dtc = dtc.fit(train_data)
dtcs

defaultdict(None,
            {'c#': DecisionTreeClassificationModel: uid=DecisionTreeClassifier_09b5e681b8f2, depth=0, numNodes=1, numClasses=2, numFeatures=100,
             'java': DecisionTreeClassificationModel: uid=DecisionTreeClassifier_8ee8926517b1, depth=0, numNodes=1, numClasses=2, numFeatures=100,
             'php': DecisionTreeClassificationModel: uid=DecisionTreeClassifier_fffd15107491, depth=2, numNodes=5, numClasses=2, numFeatures=100,
             'javascript': DecisionTreeClassificationModel: uid=DecisionTreeClassifier_9e268457b8e0, depth=0, numNodes=1, numClasses=2, numFeatures=100,
             'android': DecisionTreeClassificationModel: uid=DecisionTreeClassifier_9964cbbe039e, depth=2, numNodes=5, numClasses=2, numFeatures=100,
             'jquery': DecisionTreeClassificationModel: uid=DecisionTreeClassifier_d8f607157398, depth=2, numNodes=5, numClasses=2, numFeatures=100,
             'c++': DecisionTreeClassificationModel: uid=DecisionTreeClassifier_35895a946891

In [20]:
# pred = dtc.transform(validation_data)
# pred.select("javascript", "prediction").show(3)

In [21]:
# evaluator=MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="javascript")
# acc = evaluator.evaluate(pred)
 
# print("Prediction Accuracy: ", acc)
 
# y_pred=pred.select("prediction").collect()
# y_orig=pred.select("javascript").collect()

In [23]:
preds = defaultdict()
for tag in mostUsedTags:
    if tag not in preds:
        preds[tag] = dtcs[tag].transform(validation_data)
preds

defaultdict(None,
            {'c#': DataFrame[Id: string, Title: string, Tags: string, transformed_tfidf: array<string>, rawFeatures: vector, features: vector, cleantags: array<string>, c#: int, java: int, php: int, javascript: int, android: int, jquery: int, c++: int, python: int, iphone: int, aspnet: int, mysql: int, html: int, net: int, ios: int, objective-c: int, sql: int, css: int, linux: int, ruby-on-rails: int, windows: int, c: int, sql-server: int, ruby: int, wpf: int, xml: int, ajax: int, database: int, regex: int, windows-7: int, aspnet-mvc: int, xcode: int, django: int, osx: int, arrays: int, vbnet: int, eclipse: int, json: int, facebook: int, ruby-on-rails-3: int, ubuntu: int, performance: int, networking: int, string: int, multithreading: int, winforms: int, security: int, aspnet-mvc-3: int, visual-studio-2010: int, bash: int, homework: int, rawPrediction: vector, probability: vector, prediction: double],
             'java': DataFrame[Id: string, Title: string, Tags: str

# Subject titles to Word2Vec

In [22]:
# use df as your dataframe and add your column to it 